In [32]:
from pyspark.sql.functions import col
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import DoubleType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

In [6]:
file_path = "E:\\Cerin_big-data\\"
data = spark.read.option("delimiter", ";").option("header", "true").csv(file_path + "*.csv")
data.schema.fields

[StructField(Data,StringType,true),
 StructField(Phase A Current,StringType,true),
 StructField(Phase B Current,StringType,true),
 StructField(Phase C Current,StringType,true),
 StructField(Current Total,StringType,true),
 StructField(Power Factor,StringType,true),
 StructField(Frequency,StringType,true),
 StructField(Apparent Power,StringType,true),
 StructField(Active Power,StringType,true),
 StructField(Reactive Power,StringType,true),
 StructField(Power Production,StringType,true),
 StructField(Phase AB Voltage,StringType,true),
 StructField(Phase BC Voltage,StringType,true),
 StructField(Phase CA Voltage,StringType,true)]

In [7]:
#sqlContext = SQLContext(spark)
#sqlContext.registerDataFrameAsTable(data, "myTab")
#res = sqlContext.sql("SELECT * FROM myTab")
#res.show()
#data = data.withColumn(column, data[column].cast(DoubleType()))
#data = data.withColumn(column, str(data[column].replace(',', '.')))
#data = data.where(data[column] > 10)

In [8]:
def show_info(column, percentil, last_size , current_size):
    print("Column: {:>7} \t Percentile: {:>18} \t Before/Current/Removed: {:>8} / {:>8} / {:>5}"
        .format(column, percentil, last_size, current_size, last_size - current_size))

In [45]:
def filter_data(data):
    last_size = data.count()
    colums = [k for k in data.schema.names if 'data' not in k.lower()]
    for column in colums:        
        data = data.filter(~col(column).isin(['Invalid Data', 'I/O Timeout']))        
        commaToDot = udf(lambda x : float(str(x).replace(',', '.')), FloatType())
        data=data.withColumn(column,commaToDot(data[column]))
        percentil = data.approxQuantile(column, [0.9], 0.5)
        print(column, percentil)        
        #current_size = data.count()
        #show_info(column, percentil, last_size, current_size)        
        #last_size = current_size
    return data	

In [46]:
filtered = filter_data(data)
filtered.show()

Phase A Current [17.484657287597656]
Phase B Current [17.444656372070312]
Phase C Current [17.5040340423584]
Current Total [52.240501403808594]
Power Factor [-72.7013168334961]
Frequency [60.10990905761719]
Apparent Power [12069.9736328125]
Active Power [12069.935546875]
Reactive Power [62.91889572143555]
Power Production [39354400.0]
Phase AB Voltage [400.58013916015625]
Phase BC Voltage [400.38787841796875]
Phase CA Voltage [400.38604736328125]
+------------------+---------------+---------------+---------------+-------------+------------+---------+--------------+------------+--------------+----------------+----------------+----------------+----------------+
|              Data|Phase A Current|Phase B Current|Phase C Current|Current Total|Power Factor|Frequency|Apparent Power|Active Power|Reactive Power|Power Production|Phase AB Voltage|Phase BC Voltage|Phase CA Voltage|
+------------------+---------------+---------------+---------------+-------------+------------+---------+----------